In [1]:
import geopandas as gpd
from shapely.geometry import Polygon
import pandas as pd
import pharmada.data as data
import pharmada.customers as cu
import csv
import folium as fl
import statistics as stat

regkey = "Kreisfreie Stadt Würzburg"

In [2]:
data = data.Data(regkey)

In [3]:
"""Since the data consists of 100m x 100m (= 1 hectare) grid cells, we need to find all grid cells within the area of interest.
This is achieved by filtering the grid cells by the area's bounding box. Subsequently, all grid cells outside of the area's actual geometry are omitted.
The grid cells are defined by their center point, so all calculations revolve around it ("mp" from German "Mittelpunkt" in data & code)."""

# Calculate the center point of the southwest and northeast corner cells defining the area's bounding box
area_geom = data.AreaGeometry.geometry
area_geom.to_crs(
    epsg=3035, inplace=True
)  # EPSG:3035 is a metric projection, so we can use the coordinates as meters instead of degrees
bounds = area_geom.total_bounds

# Round the coordinates to the nearest 100m and add 50m to the southwest and subtract 50m from the northeast corner
e_sw = int(bounds[0]) // 10**2 * 10**2 + 50
n_sw = int(bounds[1]) // 10**2 * 10**2 + 50
e_ne = int(bounds[2]) // 10**2 * 10**2 - 50
n_ne = int(bounds[3]) // 10**2 * 10**2 - 50


# Check if a grid cell is within the area's bounding box
def in_range(x, y):
    return n_sw <= y <= n_ne and e_sw <= x <= e_ne


# Load the grid cells and filter them by the area's bounding box
with open("./data/Zensus_Bevoelkerung_100m-Gitter.csv", "r", newline="") as file:
    reader = csv.reader(file, delimiter=";")
    next(reader)  # skip header

    gridcells = []
    for row in reader:
        if in_range(int(row[1]), int(row[2])):
            gridcells.append(row)

# Convert to a DataFrame, clean columns and types
gridcells = pd.DataFrame(gridcells)
gridcells.rename(columns={0: "id", 1: "x_mp", 2: "y_mp", 3: "population"}, inplace=True)
gridcells["x_mp"] = gridcells["x_mp"].astype(int)
gridcells["y_mp"] = gridcells["y_mp"].astype(int)
gridcells["population"] = gridcells["population"].astype(int)

# Add cell geometries (100mx100m square around centroid)and convert to a GeoDataFrame
gridcells["geometry"] = gridcells.apply(
    lambda row: Polygon(
        [
            (row["x_mp"] - 50, row["y_mp"] - 50),
            (row["x_mp"] + 50, row["y_mp"] - 50),
            (row["x_mp"] + 50, row["y_mp"] + 50),
            (row["x_mp"] - 50, row["y_mp"] + 50),
        ]
    ),
    axis=1,
)

gridcells = gpd.GeoDataFrame(gridcells, geometry="geometry")
gridcells.set_crs(epsg=3035, inplace=True)

# Drop all grid cells that are either empty or still outside of the area's geometry
to_drop = []
for index, row in gridcells.iterrows():
    if row["population"] <= 0:
        to_drop.append(index)
    elif not area_geom.contains(row.geometry)[0]:
        to_drop.append(index)

gridcells.drop(to_drop, inplace=True)
gridcells.reset_index(drop=True, inplace=True)

In [9]:
import math
import random

# suppress FutureWarnings due to an upcoming change in GeoPandas (which will also necessitate a change in the code)
import warnings

warnings.filterwarnings("ignore", category=FutureWarning)

cell_customers = []
precise_geom = data.AreaGeometry.precise_geometry

# Calculate the relative population deviation between the 2011 Zensus data used in the grid and the 2021 projection from the regkeys data for the area.
# This is used to scale the demand for each grid cell according to population changes
scale_factor = data.RegKey.population / gridcells["population"].sum()

gridcells["population"] = gridcells.apply(
    lambda row: row["population"] * scale_factor, axis=1
)
total_population = gridcells["population"].sum()

# determine the number of customers to generate
demand = cu.get_demand(data.AreaGeometry)


for id, row in gridcells.iterrows():
    expected_customers = (row["population"] / total_population) * demand

    decimal_part, int_part = math.modf(expected_customers)

    actual_customers = int_part

    # additional customer with the probability of decimal_part
    if random.random() < decimal_part:
        actual_customers += 1

    cell_customers.append(int(actual_customers))

# set 0 for all list entries with NaN
cell_customers = [0 if math.isnan(x) else x for x in cell_customers]

customers = gridcells.sample_points(cell_customers)
# customers.explode(ignore_index=True)

# remove customers outside the area
# customers = customers[customers.within(precise_geom)]

# create a geodataframe with the customers
output = gpd.GeoDataFrame(geometry=customers)

In [15]:
customers.head(100)

0                                                  None
1                                                  None
2                                                  None
3                       POINT (4318352.821 2956064.333)
4                                                  None
                            ...                        
95                                                 None
96    MULTIPOINT (4318735.113 2956958.142, 4318755.0...
97    MULTIPOINT (4318801.748 2956917.983, 4318802.6...
98                      POINT (4318926.403 2956903.491)
99                                                 None
Name: sampled_points, Length: 100, dtype: geometry

In [ ]:
import geopandas as gpd
import numpy as np

demand = cu.get_demand(data.AreaGeometry)

# Calculate the total population
total_pop = data.RegKey.population

# Calculate probability for each grid cell
gridcells["probability"] = gridcells.apply(lambda row: row[2] / total_pop)

# Calculate the expected number of customers for each grid cell
gridcells["expected_customers"] = (
    (gridcells["probability"] * demand).round().astype(int)
)

# In case due to rounding, we generate more or fewer points than A, adjust the difference
diff = demand - gridcells["expected_customers"].sum()
while diff != 0:
    if diff > 0:
        idx = np.random.choice(gridcells.index, size=diff, p=gridcells["probability"])
        gridcells.loc[idx, "expected_customers"] += 1
        diff -= diff
    else:
        idx = np.random.choice(gridcells.index, size=-diff, p=gridcells["probability"])
        gridcells.loc[idx, "expected_customers"] -= 1
        diff += diff

# Generate the customer locations
all_samples = []
for _, row in gridcells.iterrows():
    samples = row["geometry"].sample(row["expected_customers"])
    all_samples.extend(samples)

# Convert the list of sample points into a GeoDataFrame
customers_gdf = gpd.GeoDataFrame(geometry=all_samples, crs=gridcells.crs)